In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import pandas as pd

In [ ]:
news_topic = pd.read_csv('C:/Users/cyd94/Desktop/pyCODE/미성년자 무면허 사고 뉴스 기사.csv', encoding='cp949')

In [ ]:
news_topic

,Unnamed: 0,제목,미리보기
0,0,장제원 아들 래퍼 노엘 무면허 운전경찰 폭행에 실형 선고 가능성도?,즉 집행유예 기간에 다시 무면허 교통사고를 일으킨 것이다 이 때문에 이번 사건으로 ...
1,1,장제원 아들 노엘 이번엔 무면허운전경찰관 폭행으로 체포,노엘과 접촉사고를 낸 다른 차주의 부상 상태는 가벼운 것으로 전해졌다 노엘은 무면허...
2,2,길었던 추석 연휴 전남 곳곳 사건사고로 얼룩,형사미성년자라 처벌을 피하게 됐지만 함께 타고 있던 일행 3명은 15 18 20세로...
3,3,장제원 아들 노엘 무면허 운전경찰 폭행잇단 논란에 퇴출 촉구까지,지난 19일 경찰에 따르면 서울 서초경찰서는 도로교통법상 무면허 운전 음주측정불응 ...
4,4,장제원 아들 노엘 현행범 체포됐다 무면허 운전 후 경찰 폭행,장제원 국민의힘 의원의 아들이자 래퍼 노엘21본명 장용준이 무면허 운전 후 사고를 ...
...,...,...,...
1251,1251,특급신인 다르빗슈 벌써부터 말썽,구단 및 일본 야구계는 현재 18살 미성년자로 아직 고등학교를 졸업하지 않은 다르빗...
1252,1252,외국계 기업 사규 무섭네,음주 과속난폭 뺑소니 무면허 운전 등 4대 악질 운전습관으로 사고를 내면 바로 해고...
1253,1253,2019 가요결산 승리게이트정준영최종훈김건모 성폭행범죄와의 전쟁,마약 성범죄 학교폭력 음원 사재기 조작 도박 등 각종 사건 사고로 가요계가 전체가 ...
1254,1254,장제원 아들 래퍼 노엘 집유 기간 음주측정거부경찰 폭행현행범 체포,일진설과 미성년자 성매매 시도 의혹 등에 휩싸이며 자진 하차했다 2018년 엠넷 쇼...


In [ ]:
import re

In [ ]:
def clean_html(x):
  x = re.sub("\&\w*\;","",x)
  x = re.sub("<.*?>","",x)
  return x

news_topic['제목'] = news_topic['제목'].apply(lambda x: clean_html(x))
news_topic['미리보기'] = news_topic['미리보기'].apply(lambda x: clean_html(x))

In [ ]:
total_text=""
for i in range(0, len(news_topic)):
    text1 = str(news_topic['제목'].loc[i])
    text2 = str(news_topic['미리보기'].loc[i])
    total_text = total_text + text1 + text2

In [ ]:
#pip install konlpy

In [ ]:
from konlpy.tag import Okt
Okt = Okt()
pos = Okt.pos(total_text)

In [ ]:
#ad =[t[0] for t in pos if t[1]=='Adjective']
#n =[t[0] for t in pos if t[1]=='Noun']

In [ ]:
ad=[]
n=[]

for t in pos:
    if t[1]=="Adjective":
        ad.append(t[0])
    elif t[1]=="Noun":
        n.append(t[0])

In [ ]:
morph = ad + n

In [ ]:
k_stopword = open("C:/Users/cyd94/Desktop/pyCODE/stopword_list.txt",'r',encoding='utf-8')
stopwords = list(k_stopword)

morph_stop = [txt for txt in morph if txt not in stopwords]

In [ ]:
#!pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
from gensim import corpora
import logging
from gensim.models.callbacks import PerplexityMetric
from gensim.models.callbacks import CoherenceMetric

In [ ]:
if __name__ == '__main__':
  processed_data = [sent.strip().split(",") for sent in morph_stop]
  dictionary = corpora.Dictionary(processed_data)
  dictionary.filter_extremes(no_below=10, no_above=0.05)
  corpus = [dictionary.doc2bow(text) for text in processed_data]
  print('Number of unique tokens: %d' % len(dictionary))
  print('Number of documents: %d' % len(corpus))
  
  logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
  
  perplexity_logger = PerplexityMetric(corpus=corpus, logger='shell')
  coherence_logger = CoherenceMetric(corpus=corpus, coherence="u_mass", logger='shell')
  lda_model = LdaModel(corpus, id2word=dictionary, num_topics=10, passes=30, callbacks=[coherence_logger, perplexity_logger])
  topics = lda_model.print_topics(num_words=5)
  
  for topic in topics:
    print(topic)

2021-09-28 11:11:19,230 : INFO : using symmetric alpha at 0.1
2021-09-28 11:11:19,231 : INFO : using symmetric eta at 0.1
2021-09-28 11:11:19,234 : INFO : using serial LDA version on this node
2021-09-28 11:11:19,238 : INFO : running online (multi-pass) LDA training, 10 topics, 30 passes over the supplied corpus of 40684 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2021-09-28 11:11:19,241 : INFO : PROGRESS: pass 0, at document #2000/40684


Number of unique tokens: 682
Number of documents: 40684


2021-09-28 11:11:19,836 : INFO : merging changes from 2000 documents into a model of 40684 documents
2021-09-28 11:11:19,847 : INFO : topic #5 (0.100): 0.325*"없다" + 0.126*"있으며" + 0.108*"어떻게" + 0.108*"심각한" + 0.054*"있는" + 0.036*"있다" + 0.036*"아닌" + 0.036*"있었다" + 0.036*"의한" + 0.018*"없어"
2021-09-28 11:11:19,849 : INFO : topic #2 (0.100): 0.252*"인한" + 0.230*"있어" + 0.153*"있습니다" + 0.077*"있던" + 0.055*"있다는" + 0.033*"어떻게" + 0.033*"없어" + 0.022*"손쉽게" + 0.022*"어렵다고" + 0.022*"있으며"
2021-09-28 11:11:19,851 : INFO : topic #7 (0.100): 0.489*"있다" + 0.168*"없는" + 0.054*"아니라" + 0.054*"많은" + 0.049*"없어" + 0.035*"아니다" + 0.030*"의한" + 0.025*"무서운" + 0.020*"어떻게" + 0.015*"있었다"
2021-09-28 11:11:19,855 : INFO : topic #9 (0.100): 0.321*"있다" + 0.165*"같은" + 0.097*"있는" + 0.088*"있고" + 0.078*"어렵다고" + 0.068*"철저히" + 0.044*"있다는" + 0.029*"어떻게" + 0.029*"의한" + 0.019*"있다고"
2021-09-28 11:11:19,857 : INFO : topic #3 (0.100): 0.401*"같은" + 0.097*"인한" + 0.097*"인해" + 0.076*"있어" + 0.076*"다행히" + 0.043*"정확한" + 0.033*"있는" + 0.033*"아닌" + 0.0

2021-09-28 11:11:22,791 : INFO : topic #7 (0.100): 0.160*"있다" + 0.056*"없는" + 0.019*"아니라" + 0.019*"많은" + 0.017*"없어" + 0.012*"아니다" + 0.011*"의한" + 0.009*"무서운" + 0.007*"어떻게" + 0.006*"있었다"
2021-09-28 11:11:22,793 : INFO : topic #4 (0.100): 0.151*"대학생" + 0.057*"있다" + 0.047*"씨" + 0.046*"보드" + 0.029*"없는" + 0.019*"입니다" + 0.017*"있습니다" + 0.016*"없다" + 0.013*"있었던" + 0.008*"있는"
2021-09-28 11:11:22,795 : INFO : topic #1 (0.100): 0.166*"군" + 0.135*"등" + 0.126*"세" + 0.122*"경찰" + 0.120*"것" + 0.080*"운전자" + 0.074*"이" + 0.072*"청소년" + 0.046*"건" + 0.012*"수"
2021-09-28 11:11:22,797 : INFO : topic #8 (0.100): 0.225*"렌터카" + 0.220*"차량" + 0.153*"명" + 0.151*"처벌" + 0.138*"사망" + 0.073*"발생" + 0.007*"경찰" + 0.002*"어떻게" + 0.001*"손쉽게" + 0.001*"정확한"
2021-09-28 11:11:22,799 : INFO : topic #2 (0.100): 0.349*"차" + 0.191*"음주운전" + 0.107*"지난" + 0.050*"최근" + 0.042*"형사" + 0.040*"수" + 0.017*"살" + 0.016*"날" + 0.009*"뺑소니" + 0.008*"인한"
2021-09-28 11:11:22,801 : INFO : topic diff=0.515184, rho=0.353553
2021-09-28 11:11:22,803 : INFO :

2021-09-28 11:11:26,568 : INFO : topic #0 (0.100): 0.099*"킥" + 0.072*"카셰어링" + 0.069*"안전" + 0.063*"이용" + 0.062*"고등학생" + 0.062*"혐의" + 0.057*"이상" + 0.052*"사건" + 0.052*"질주" + 0.049*"확인"
2021-09-28 11:11:26,571 : INFO : topic #2 (0.100): 0.344*"차" + 0.184*"지난" + 0.148*"음주운전" + 0.127*"최근" + 0.096*"날" + 0.003*"형사" + 0.003*"수" + 0.001*"살" + 0.001*"뺑소니" + 0.001*"인한"
2021-09-28 11:11:26,574 : INFO : topic #6 (0.100): 0.189*"모두" + 0.169*"중학생" + 0.112*"상자" + 0.091*"중고생" + 0.079*"준" + 0.072*"관련" + 0.068*"오전" + 0.049*"차로" + 0.038*"크게" + 0.025*"현장"
2021-09-28 11:11:26,579 : INFO : topic #5 (0.100): 0.272*"무면허" + 0.244*"사고" + 0.236*"미성년" + 0.159*"운전" + 0.078*"교통사고" + 0.000*"없다" + 0.000*"있으며" + 0.000*"어떻게" + 0.000*"심각한" + 0.000*"있는"
2021-09-28 11:11:26,581 : INFO : topic diff=0.095100, rho=0.258199
2021-09-28 11:11:26,583 : INFO : PROGRESS: pass 0, at document #32000/40684
2021-09-28 11:11:26,978 : INFO : merging changes from 2000 documents into a model of 40684 documents
2021-09-28 11:11:26,982 : INFO

...

2021-09-28 11:21:03,825 : INFO : topic #8 (0.100): 0.226*"명" + 0.212*"차량" + 0.154*"렌터카" + 0.137*"처벌" + 0.090*"사망" + 0.083*"발생" + 0.062*"어떻게" + 0.000*"경우" + 0.000*"음주" + 0.000*"없다"
2021-09-28 11:21:03,827 : INFO : topic #6 (0.100): 0.322*"없는" + 0.156*"있는" + 0.071*"입니다" + 0.071*"없어" + 0.061*"모두" + 0.045*"중학생" + 0.042*"관련" + 0.039*"차로" + 0.030*"상자" + 0.028*"준"
2021-09-28 11:21:03,829 : INFO : topic diff=0.236206, rho=0.140940
2021-09-28 11:21:03,831 : INFO : PROGRESS: pass 29, at document #4000/40684
2021-09-28 11:21:04,126 : INFO : merging changes from 2000 documents into a model of 40684 documents
2021-09-28 11:21:04,130 : INFO : topic #1 (0.100): 0.165*"등" + 0.129*"경찰" + 0.108*"군" + 0.106*"건" + 0.105*"이" + 0.102*"것" + 0.102*"운전자" + 0.092*"세" + 0.063*"청소년" + 0.000*"음주"
2021-09-28 11:21:04,131 : INFO : topic #5 (0.100): 0.278*"무면허" + 0.250*"미성년" + 0.233*"사고" + 0.165*"운전" + 0.061*"교통사고" + 0.000*"음주" + 0.000*"경우" + 0.000*"아들" + 0.000*"노엘" + 0.000*"당시"
2021-09-28 11:21:04,134 : INFO : topic

2021-09-28 11:21:06,208 : INFO : topic #4 (0.100): 0.076*"안성" + 0.073*"및" + 0.069*"씨" + 0.065*"오토바이" + 0.064*"면허" + 0.063*"국민" + 0.062*"보드" + 0.058*"업체" + 0.053*"승용차" + 0.053*"보험"
2021-09-28 11:21:06,209 : INFO : topic diff=0.031293, rho=0.140940
2021-09-28 11:21:06,211 : INFO : PROGRESS: pass 29, at document #18000/40684
2021-09-28 11:21:06,497 : INFO : merging changes from 2000 documents into a model of 40684 documents
2021-09-28 11:21:06,500 : INFO : topic #5 (0.100): 0.302*"무면허" + 0.254*"사고" + 0.226*"미성년" + 0.150*"운전" + 0.057*"교통사고" + 0.000*"경우" + 0.000*"음주" + 0.000*"사람" + 0.000*"당시" + 0.000*"불법"
2021-09-28 11:21:06,503 : INFO : topic #4 (0.100): 0.101*"안성" + 0.066*"승용차" + 0.066*"씨" + 0.066*"국민" + 0.063*"면허" + 0.063*"오토바이" + 0.063*"및" + 0.062*"업체" + 0.057*"보드" + 0.053*"도로"
2021-09-28 11:21:06,505 : INFO : topic #7 (0.100): 0.123*"있다" + 0.067*"무면허운전" + 0.056*"뒤" + 0.054*"피해" + 0.052*"도주" + 0.050*"결과" + 0.049*"절도" + 0.049*"고교생" + 0.048*"그" + 0.046*"청와대"
2021-09-28 11:21:06,508 : INFO

2021-09-28 11:21:08,919 : INFO : topic diff=0.033425, rho=0.140940
2021-09-28 11:21:08,921 : INFO : PROGRESS: pass 29, at document #32000/40684
2021-09-28 11:21:09,271 : INFO : merging changes from 2000 documents into a model of 40684 documents
2021-09-28 11:21:09,274 : INFO : topic #1 (0.100): 0.152*"등" + 0.140*"군" + 0.132*"경찰" + 0.110*"세" + 0.106*"이" + 0.106*"것" + 0.094*"운전자" + 0.078*"청소년" + 0.059*"건" + 0.000*"경우"
2021-09-28 11:21:09,276 : INFO : topic #0 (0.100): 0.078*"킥" + 0.062*"카셰어링" + 0.061*"범죄" + 0.059*"미만" + 0.056*"확인" + 0.056*"사건" + 0.055*"혐의" + 0.054*"이상" + 0.049*"이용" + 0.048*"고등학생"
2021-09-28 11:21:09,279 : INFO : topic #8 (0.100): 0.215*"렌터카" + 0.213*"차량" + 0.205*"명" + 0.131*"처벌" + 0.120*"사망" + 0.085*"발생" + 0.005*"어떻게" + 0.000*"경우" + 0.000*"부모" + 0.000*"음주"
2021-09-28 11:21:09,281 : INFO : topic #5 (0.100): 0.282*"무면허" + 0.246*"사고" + 0.239*"미성년" + 0.153*"운전" + 0.069*"교통사고" + 0.000*"경우" + 0.000*"부모" + 0.000*"음주" + 0.000*"불법" + 0.000*"사람"
2021-09-28 11:21:09,284 : INFO : to

2021-09-28 11:21:11,509 : INFO : CorpusAccumulator accumulated stats from 12000 documents
2021-09-28 11:21:11,514 : INFO : CorpusAccumulator accumulated stats from 13000 documents
2021-09-28 11:21:11,520 : INFO : CorpusAccumulator accumulated stats from 14000 documents
2021-09-28 11:21:11,529 : INFO : CorpusAccumulator accumulated stats from 15000 documents
2021-09-28 11:21:11,539 : INFO : CorpusAccumulator accumulated stats from 16000 documents
2021-09-28 11:21:11,546 : INFO : CorpusAccumulator accumulated stats from 17000 documents
2021-09-28 11:21:11,552 : INFO : CorpusAccumulator accumulated stats from 18000 documents
2021-09-28 11:21:11,561 : INFO : CorpusAccumulator accumulated stats from 19000 documents
2021-09-28 11:21:11,571 : INFO : CorpusAccumulator accumulated stats from 20000 documents
2021-09-28 11:21:11,580 : INFO : CorpusAccumulator accumulated stats from 21000 documents
2021-09-28 11:21:11,588 : INFO : CorpusAccumulator accumulated stats from 22000 documents
2021-09-28

(0, '0.081*"사건" + 0.078*"카셰어링" + 0.072*"혐의" + 0.069*"범죄" + 0.064*"확인"')
(1, '0.177*"등" + 0.122*"경찰" + 0.114*"군" + 0.105*"이" + 0.102*"운전자"')
(2, '0.305*"차" + 0.225*"음주운전" + 0.161*"지난" + 0.127*"최근" + 0.068*"날"')
(3, '0.454*"수" + 0.243*"대여" + 0.148*"청원" + 0.019*"같은" + 0.000*"경우"')
(4, '0.097*"씨" + 0.093*"및" + 0.090*"오토바이" + 0.075*"안성" + 0.070*"면허"')
(5, '0.271*"무면허" + 0.253*"미성년" + 0.229*"사고" + 0.169*"운전" + 0.066*"교통사고"')
(6, '0.152*"모두" + 0.112*"중학생" + 0.105*"관련" + 0.096*"차로" + 0.077*"상자"')
(7, '0.126*"무면허운전" + 0.089*"뒤" + 0.078*"도주" + 0.066*"그" + 0.061*"피해"')
(8, '0.241*"명" + 0.227*"차량" + 0.165*"렌터카" + 0.146*"처벌" + 0.096*"사망"')
(9, '0.187*"형사" + 0.119*"상태" + 0.116*"촉법소년" + 0.107*"조사" + 0.099*"뺑소니"')


In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_data, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score (c_v): ', coherence_lda)

2021-09-28 11:21:22,356 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate probabilities from sliding windows
2021-09-28 11:21:33,785 : INFO : 7 accumulators retrieved from output queue
2021-09-28 11:21:33,815 : INFO : accumulated word occurrence stats for 20532 virtual documents



Coherence Score (c_v):  0.8319530749862855


In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_data, dictionary=dictionary, coherence="u_mass")
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score (u_mass): ', coherence_lda)

2021-09-28 11:21:35,073 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2021-09-28 11:21:35,079 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2021-09-28 11:21:35,087 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2021-09-28 11:21:35,093 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2021-09-28 11:21:35,099 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2021-09-28 11:21:35,108 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2021-09-28 11:21:35,120 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2021-09-28 11:21:35,128 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2021-09-28 11:21:35,134 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2021-09-28 11:21:35,144 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2021-09-28 11:21:35,151 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2021-09-28 11:21:35


Coherence Score (u_mass):  -21.743969065051594


In [ ]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis
import os 
os.environ["PYTHONIOENCODING"] ="utf-8"
pyLDAvis.enable_notebook()

C:\Users\cyd94\miniconda3\envs\multu\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
lda_vis = gensimvis.prepare(lda_model,corpus,dictionary)
pyLDAvis.display(lda_vis)

C:\Users\cyd94\miniconda3\envs\multu\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\cyd94\miniconda3\envs\multu\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \
